In [10]:
import numpy as np
import glob
import os
import shutil
import cv2
from annoy import AnnoyIndex
import torch

# Important so PyTorch can load LSTM Model Weights
from Utils.PTModel.Models import LSTMModel

MODELNAME = "AutoEncoderwATT"
NUMBEROFLEVELS = 150
RANDOMSEED = 1

In [11]:
gameName = "LodeRunner"
rowLength = 32
numOfRows = 22

lrEmbeddingPath = f"Models/{MODELNAME}/LevelUnifiedRep/{gameName}"
lrEmbeddingPaths = sorted(glob.glob(f"{lrEmbeddingPath}/Level*.npy"))
lrStringNames = [path[path.rfind("/")+1:path.rfind("Embedding.npy")] for path in lrEmbeddingPaths]

columnRefArray = np.array([np.arange(0, 32) for i in range(numOfRows+5)]).flatten()

In [12]:
model = torch.load(f"Models/{MODELNAME}/LodeRunnerLSTM.pt")

/tmp/ipykernel_29725/2861288654.py:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model = torch.load(f"Models/{MODELNAME}/LodeRunnerLSTM.pt")


In [13]:
# Deletes the directory of the lode runner levels folder
shutil.rmtree(f"Models/{MODELNAME}/LodeRunnerLevels")

In [14]:
# To safeguard against accidentally deleting the wrong model folder
if os.path.isdir(f"Models/{MODELNAME}/LodeRunnerLevels"):
    raise RuntimeError(f"Models/{MODELNAME}/LodeRunnerLevels path already exists. Please delete it an re-run the code.")

os.mkdir(f"Models/{MODELNAME}/LodeRunnerLevels")
os.mkdir(f"Models/{MODELNAME}/LodeRunnerLevels/images")
os.mkdir(f"Models/{MODELNAME}/LodeRunnerLevels/txts")

device = "cuda" if torch.cuda.is_available() else "cpu"

model.to(device)
model.eval()

tileMap = np.load(f"{lrEmbeddingPath}/centerTiles.npy")
embedMap = np.load(f"{lrEmbeddingPath}/embeddings.npy")

print(f"tileMap shape: {tileMap.shape}")
print(f"embedMap shape: {embedMap.shape}")

print("Maps Loaded")

nnTree = AnnoyIndex(256, 'euclidean')

print("Initialised Tree")

#for i in range(embedMap.shape[0]): print(f"Added {i}th item to tree"), nnTree.add_item(i, embedMap[i])
for i in range(embedMap.shape[0]): nnTree.add_item(i, embedMap[i])

print("Building Tree")
nnTree.build(15)
print("Tree Built")

print("Annoy Map Trained")

# Reading in sprites to compare and make string form of level
lrSpritePaths = glob.glob("/home/surfytom/Projects/Dissertation/Repos/TileEmbeddingDissertation/data/tomData/sprites/loderunner/*.png")
lrSprites = {path[path.rfind("/")+1:path.rfind(".png")]: cv2.cvtColor(cv2.imread(path), cv2.COLOR_BGR2RGB) for path in lrSpritePaths}

padSize = rowLength * 3 # lode runner row length * 3 as paper uses previous 3 rows for lstm
N = padSize

np.random.seed(RANDOMSEED)
randStartPoint = np.random.randint(0, len(lrEmbeddingPaths)-NUMBEROFLEVELS) if len(lrEmbeddingPaths)-NUMBEROFLEVELS > 0 else 0
print(f"Random Start Point: {randStartPoint}")

for i, levelEmbeddingPath in enumerate(lrEmbeddingPaths[randStartPoint:randStartPoint+NUMBEROFLEVELS]):

    input = []
    column = []
    target = []

    levelEmbeddingArray = np.load(levelEmbeddingPath)

    for j in range(len(levelEmbeddingArray) - N):

        padLength = (N - j) if j < N else 0
        RowCutOff = 0 if j <= N else RowCutOff+1

        dataI = np.concatenate((np.zeros(shape=(padLength, 256)), levelEmbeddingArray[RowCutOff:j]), axis=0)

        dataT = levelEmbeddingArray[j:j+N]

        levelIdx = np.concatenate((np.zeros(shape=(padLength)), columnRefArray[RowCutOff:j]), axis=0)
        dataC = np.zeros(shape=(N, 256))
        for t in range(N): dataC[t][int(levelIdx[t])] = 1

        column.append(dataC)
        input.append(dataI)
        target.append(dataT)

    input = np.array(input)
    target = np.array(target)
    column = np.array(column)

    # print(f"Input shape: {input.shape}")
    # print(f"target shape: {target.shape}")
    # print(f"column shape: {column.shape}")

    #levelImage = np.empty(shape=((7 * 3) * 16, 32 * 16, 3))
    levelImage = np.empty(shape=((22) * 16, 32 * 16, 3))

    for b in range(8):
    
        if b == 0:
            inputTensor = torch.tensor(input[b*N], dtype=torch.float32).to(device)
        else:
            inputTensor = torch.tensor(previousCol, dtype=torch.float32).to(device)
        
        # print(f"input size: {inputTensor.size()}")
        
        if b == 7:
            targetTensor = torch.tensor(target[-33], dtype=torch.float32).to(device)
            columnTensor = torch.tensor(column[-33], dtype=torch.float32).to(device)
        else:
            targetTensor = torch.tensor(target[b*N], dtype=torch.float32).to(device)
            columnTensor = torch.tensor(column[b*N], dtype=torch.float32).to(device)
        
        # print(f"target size: {targetTensor.size()}")
        # print(f"column size: {columnTensor.size()}")

        # print("Running Model")

        nextCol = model(inputTensor, targetTensor, columnTensor)
        nextCol = nextCol.cpu().detach().numpy()
        # print(f"next col size: {nextCol.shape}")

        previousCol = nextCol
        nextCol = nextCol.reshape(3, 32, nextCol.shape[1])

        if b == 7:
            nextCol = nextCol[0, :, :].reshape(1, 32, 256)

        partLevelImage = np.zeros(shape=(nextCol.shape[0] * 16, nextCol.shape[1] * 16, 3), dtype=np.uint8)

        for t, row in enumerate(nextCol):
            for j, embedding in enumerate(row):

                nearestEmbedding = nnTree.get_nns_by_vector(embedding, 10, search_k=-1, include_distances=False)
                
                #print(np.random.randint(0, len(nearestEmbedding)))
                #nearestEmbedding = nearestEmbedding[np.random.randint(0, len(nearestEmbedding))]
                tileImage = tileMap[nearestEmbedding[0]]
                #print(f"Tile Image Shape: {tileImage.shape}")
                partLevelImage[t*16 : t*16+16, j*16 : j*16+16, :] = tileImage
        
        # print(f"Part Level Image Shape: {partLevelImage.shape}")
        if b == 7:
            levelImage[b*3*16:((b*3)+1)*16, :, :] = partLevelImage
        else:
            levelImage[b*3*16:(b+1)*3*16, :, :] = partLevelImage

    # plt.figure()
    # plt.imshow(levelImage.astype(np.uint8))

    tiles = ""
    for t in range(0, levelImage.shape[0], 16):
        for p in range(0, levelImage.shape[1], 16):
            for k, v in lrSprites.items():
                if (levelImage[t:t+16, p:p+16, :] == v).all():
                    tiles += k
            
        tiles += "\n"

    cv2.imwrite(f"Models/{MODELNAME}/LodeRunnerLevels/images/{lrStringNames[i]}.png", cv2.cvtColor(levelImage.astype(np.uint8), cv2.COLOR_BGR2RGB))
    with open(f"Models/{MODELNAME}/LodeRunnerLevels/txts/{lrStringNames[i]}.txt", "w") as f:
        f.writelines(tiles)

tileMap shape: (105600, 16, 16, 3)
embedMap shape: (105600, 256)
Maps Loaded
Initialised Tree
Building Tree
Tree Built
Annoy Map Trained
Random Start Point: 0
